In [16]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from torchvision.models import resnet50  # Or your specific ResNet variant
from PIL import Image
import numpy as np
import cv2
from pytorch_grad_cam import GradCAM
from pytorch_grad_cam.utils.image import show_cam_on_image

In [17]:
# --- 1. Define Device ---
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [18]:
import tensorflow as tf

h5_file_path = 'trained_model.h5'  # Replace with the actual path to your .h5 file
try:
    model = tf.keras.models.load_model(h5_file_path)
    print("Model loaded successfully from {}".format(h5_file_path))
except FileNotFoundError:
    print("Error: Model file not found at {}".format(h5_file_path))
    exit()
except Exception as e:
    print("Error loading model: {}".format(e))
    print(e)  # Print the full exception for debugging
    exit()# --- 2. Instantiate Your ResNet Model Architecture ---
model = resnet50(pretrained=False)

Model loaded successfully from trained_model.h5


In [19]:
import torch
# --- 1. Define Device ---
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
from torchvision.models import resnet50
# --- 2. Instantiate Your ResNet Model Architecture ---
model = resnet50(pretrained=False)
# --- 4. Set Model to Evaluation Mode ---
model.eval().to(device)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [20]:
import torchvision.transforms as transforms

In [21]:
# --- 5. Define Image Transformations (Match Your Training) ---
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Adjust if your input size was different
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # ImageNet normalization if applicable
])

In [22]:
from PIL import Image

In [23]:
import torch
# --- 1. Define Device ---
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
import torchvision.transforms as transforms
from PIL import Image
# --- 5. Define Image Transformations (Match Your Training) ---
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Adjust if your input size was different
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # ImageNet normalization if applicable
])
# --- 6. Load and Preprocess the Image for Explanation ---
image_path = "D:/.vscode/My Major Project/WhatsApp Image 2025-05-03 at 12.44.31_015839af.jpg"  # Replace with the path to your image
try:
    original_image = Image.open(image_path).convert('RGB')
    input_tensor = transform(original_image).unsqueeze(0).to(device)  # Add batch dimension
except FileNotFoundError:
    print(f"Error: Image not found at {image_path}")
    exit()
except Exception as e:
    print(f"Error loading image: {e}")
    exit()

In [24]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from torchvision.models import resnet50  # Or your specific ResNet variant
from PIL import Image
import numpy as np
import cv2
from pytorch_grad_cam import GradCAM
from pytorch_grad_cam.utils.image import show_cam_on_image
# --- 2. Instantiate Your ResNet Model Architecture ---
model = resnet50(pretrained=False)
target_layer = model.layer4[-1].conv3  # Try this for ResNet50 initially


In [25]:
cam = GradCAM(model=model, target_layers=[target_layer])

In [26]:
# --- 9. Get Model Prediction (Optional, but useful for targeting a specific class) ---
output = model(input_tensor)
_, predicted_class = torch.max(output, 1)
predicted_class_index = predicted_class.item()
print(f"Predicted class index: {predicted_class_index}")

Predicted class index: 27


In [27]:
from pytorch_grad_cam.utils.model_targets import ClassifierOutputTarget

In [28]:
# --- 10. Generate the Grad-CAM Heatmap ---
# You can target a specific class or use the predicted class
targets = [ClassifierOutputTarget(predicted_class_index)]  # Create the targets list
# specific_target_class_index = your_target_class_index
# targets = [ClassifierOutputTarget(specific_target_class_index)]

grayscale_cam = cam(input_tensor=input_tensor, targets=targets)  # Use 'targets'
grayscale_cam = grayscale_cam[0, :]  # Remove batch dimension

In [29]:
# --- 11. Visualize the Heatmap on the Original Image ---
original_image_np = np.array(original_image)[:, :, ::-1]  # Convert RGB to BGR for OpenCV

# Resize the grayscale_cam to the original image size
resized_cam = cv2.resize(grayscale_cam, (original_image_np.shape[1], original_image_np.shape[0]))

visualization = show_cam_on_image(original_image_np / 255.0, resized_cam, use_rgb=True)

In [30]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from torchvision.models import resnet50  # Or your specific ResNet variant
from PIL import Image
import numpy as np
import cv2
from pytorch_grad_cam import GradCAM
from pytorch_grad_cam.utils.image import show_cam_on_image

# --- 11. Visualize the Heatmap on the Original Image ---
original_image_np = np.array(original_image)[:, :, ::-1]  # Convert RGB to BGR for OpenCV
resized_cam = cv2.resize(grayscale_cam, (original_image_np.shape[1], original_image_np.shape[0]))
visualization = show_cam_on_image(original_image_np / 255.0, resized_cam, use_rgb=True)

import cv2
# --- 12. Display or Save the Visualization ---
cv2.imshow("Grad-CAM Visualization", visualization)
cv2.waitKey(0)
cv2.destroyAllWindows()
# cv2.imwrite("gradcam_output.jpg", visualization)

# --- 13. Clean Up ---
cam.remove_hook()

AttributeError: 'GradCAM' object has no attribute 'remove_hook'